In [21]:
from decimal import Decimal
import json
import psycopg2

mid_lat, mid_long = 36.00192134954784, -78.937821464194


def rows_to_dict(rowsss, cccursor):
    columns = [desc[0] for desc in cccursor.description]
    result = []
    for rrr in rowsss:
        result.append(dict(zip(columns, rrr)))
    return result


def combine_dicts_to_dict_of_lists(dict_list):
    result = {}
    for dictionary in dict_list:
        for key, value in dictionary.items():
            if key not in result:
                result[key] = []
            result[key].append(value)
    for key in result.keys():
        if isinstance(result[key][0], Decimal):
            for iii, item in enumerate(result[key]):
                result[key][iii] = float(item)
    return result


def get_data_dict():
    connection = psycopg2.connect(database=db_name, user=db_user,
                                  password=db_password, host=db_host, port=db_port)
    cursor = connection.cursor()

    cursor.execute("SELECT latitude as latitude, longitude as longitude, "
                   "rssi as rssi, azimuth as azimuth FROM \"SigCapDetails\" "
                   "WHERE \"mPci\"=20 "#and \"uid\">2518 "
                   "and \"recordTimeStamp\" between '2023-08-01' and '2023-08-31'")
    rows = cursor.fetchall()

    df_ret = combine_dicts_to_dict_of_lists(rows_to_dict(rows, cursor))
    cursor.close()
    connection.close()
    return df_ret


with open("SQLpwd.json", 'r') as file:
    credential = json.load(file)
db_host = credential['db_host']
db_port = credential['db_port']
db_name = credential['db_name']
db_user = credential['db_user']
db_password = credential['db_password']

df_SigCapDetails = get_data_dict()
print(df_SigCapDetails['azimuth'][0:5])
print(df_SigCapDetails['latitude'][0:5])
print(df_SigCapDetails['longitude'][0:5])
print(len(list(df_SigCapDetails)))

[55.28700380270865, 155.21494239789422, 52.42495988008196, 78.01999413424838, 69.69835802688246]
[36.00112734, 36.00105809, 36.00198757, 36.00123582, 36.00207911]
[-78.93929168, -78.93896998, -78.9380465, -78.93855986, -78.93796977]
4


In [22]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np

load_dotenv()
API_KEY = os.getenv("API_KEY")

In [23]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, gmap
from bokeh.models import ColumnDataSource, LogColorMapper, LinearColorMapper, HoverTool, GMapOptions, WheelZoomTool, ArrowHead
from bokeh.models import ColorBar
import bokeh.palettes as palettes
output_notebook()

Loading BokehJS ...

In [29]:
def plot_gmap_Duke_ss(inner_field, title, df_source, mid_lat=mid_lat, mid_long=mid_long,
                      zoom=17, circle_size=4):
    df_source = pd.DataFrame(list(zip(*[df_source[kkey] for kkey in list(df_source)])),
                             columns=list(df_source))
    # df_source = pd.DataFrame(list(zip(df_source['longitude'], df_source['latitude'],
    #                                   df_source['rssi'])), columns=['longitude', 'latitude', 'rssi'])
    print(df_source.head())
    print(len(df_source['longitude']))
    inner_gmap_options = GMapOptions(
    lat=mid_lat,
    lng=mid_long,
    map_type='satellite',
    zoom=zoom
    )
    inner_hover = HoverTool(
        tooltips=[
            (inner_field, '@' + inner_field + '{0.0}'),
        ]
    )

    inner_p = gmap(
        API_KEY,
        inner_gmap_options,
        title=title,
        width=800,
        height=600,
        tools=[inner_hover, 'pan']
    )
    inner_source = ColumnDataSource(df_source)

    inner_color_mapper = LinearColorMapper(palette=palettes.inferno(20))
    inner_p.circle('longitude', 'latitude', source=inner_source,
              color={'field': inner_field, 'transform': inner_color_mapper},
              size=circle_size)
    inner_color_bar = ColorBar(color_mapper=inner_color_mapper, location=(0, 0))
    inner_p.add_layout(inner_color_bar, 'right')
    inner_p.add_tools(WheelZoomTool())
    return inner_p


p1 = plot_gmap_Duke_ss(inner_field='azimuth',
                         title='rssi SigCapDetails',
                         df_source=df_SigCapDetails, circle_size=7)
show(p1)

    latitude  longitude  rssi     azimuth
0  36.001127 -78.939292   -89   55.287004
1  36.001058 -78.938970   -91  155.214942
2  36.001988 -78.938046   -69   52.424960
3  36.001236 -78.938560   -81   78.019994
4  36.002079 -78.937970   -65   69.698358
267


In [18]:
dict_test = {'latitude': [mid_lat, mid_lat + 0.0001], 'longitude': [mid_long, mid_long + 0.0001],
             'rssi': [-40, -50]}
df_test = pd.DataFrame.from_dict(dict_test)
p2 = plot_gmap_Duke_ss(inner_field='rssi',
                         title='rssi SigCapDetails',
                         df_source=df_test, circle_size=7)
show(p2)

   longitude   latitude  rssi
0 -78.937821  36.001921   -40
1 -78.937721  36.002021   -50


In [6]:
from bokeh.models import (Arrow, Label, NormalHead, OpenHead,
                          Plot, Range1d, TeeHead, VeeHead)
from bokeh.plotting import show

ARROW_HEADS = [VeeHead]#[TeeHead, OpenHead, NormalHead, VeeHead]
HEIGHT = 35 * len(ARROW_HEADS)

p = Plot(width=150, height=HEIGHT,
         x_range=Range1d(0,1), y_range=Range1d(-0.5, len(ARROW_HEADS) - 0.5),
         toolbar_location=None, outline_line_color=None, min_border_left=0,
         min_border_right=0, min_border_top=0, min_border_bottom=0)

for i, style in enumerate(ARROW_HEADS):
    arrow = Arrow(x_start=0.2, y_start=i, x_end=0.2, y_end=i-0.1, end=style(
        line_width=2, line_color='blue', line_alpha=0.8,
                             size=10,
                             line_dash='solid'
    ))
    p.add_layout(arrow)


show(p)

In [35]:
def plot_gmap_Duke_ss_arrow_head(inner_field='rssi', title='rssi', df_source=df_SigCapDetails,
                                 mid_lat=mid_lat, mid_long=mid_long,
                      zoom=17, circle_size=4, ang_sep=10):
    # df_source = pd.DataFrame(list(zip(*[df_source[kkey] for kkey in list(df_source)])),
    #                          columns=list(df_source))
    df_source = pd.DataFrame(list(zip(#df_source['uid'],
                                      df_source['longitude'], df_source['latitude'],
                                      df_source['rssi'], df_source['azimuth'],
                                      np.zeros(len(df_source['azimuth'])),
                                      np.zeros(len(df_source['azimuth']))
                                      #np.pi/2 - (np.radians(df_source['azimuth']) + ang_sep/2),
                                      #np.pi/2 - (np.radians(df_source['azimuth']) - ang_sep/2)
                                      )
                                  ),
                             columns=[#'uid',
                                      'longitude', 'latitude', 'rssi', 'azimuth', 'azimuth_start',
                                      'azimuth_end'])
    print(df_source.head())
    # converting from
    for idx, angle in enumerate(df_source['azimuth']):
        if -90 < df_source['azimuth'][idx] < 180:
            df_source['azimuth_end'][idx] = 90 - ((df_source['azimuth'][idx]) - ang_sep/2)
            df_source['azimuth_start'][idx] = 90 - ((df_source['azimuth'][idx]) + ang_sep/2)
        if -180 < df_source['azimuth'][idx] < -90:
            df_source['azimuth_end'][idx] = -270 - ((df_source['azimuth'][idx]) - ang_sep/2)
            df_source['azimuth_start'][idx] = -270 - ((df_source['azimuth'][idx]) + ang_sep/2)

    print(df_source.head())
    inner_gmap_options = GMapOptions(
    lat=mid_lat,
    lng=mid_long,
    map_type='satellite',
    zoom=zoom
    )
    inner_hover = HoverTool(
        tooltips=[
            (inner_field, '@' + inner_field + '{0.0}'),
            #('uid', '@' + 'uid' + '{0.0}')
            ('azimuth', '@azimuth{0.0}')
        ]
    )

    inner_p = gmap(
        API_KEY,
        inner_gmap_options,
        title=title,
        width=800,
        height=600,
        tools=[inner_hover, 'pan']
    )

    inner_source = ColumnDataSource(df_source)

    inner_color_mapper = LinearColorMapper(palette=palettes.inferno(20))

    inner_p.annular_wedge(x='longitude', y='latitude', inner_radius=0, outer_radius=20,
                          start_angle='azimuth_start', end_angle='azimuth_end', source=inner_source,
                          color={'field': inner_field, 'transform': inner_color_mapper}, alpha=0.8,
                          start_angle_units='deg', end_angle_units='deg')
    inner_color_bar = ColorBar(color_mapper=inner_color_mapper, location=(0, 0))
    inner_p.add_layout(inner_color_bar, 'right')
    inner_p.add_tools(WheelZoomTool())
    return inner_p

# print(df_SigCapDetails['longitude'])
show(plot_gmap_Duke_ss_arrow_head())


   longitude   latitude  rssi     azimuth  azimuth_start  azimuth_end
0 -78.939292  36.001127   -89   55.287004            0.0          0.0
1 -78.938970  36.001058   -91  155.214942            0.0          0.0
2 -78.938046  36.001988   -69   52.424960            0.0          0.0
3 -78.938560  36.001236   -81   78.019994            0.0          0.0
4 -78.937970  36.002079   -65   69.698358            0.0          0.0
   longitude   latitude  rssi     azimuth  azimuth_start  azimuth_end
0 -78.939292  36.001127   -89   55.287004      29.712996    39.712996
1 -78.938970  36.001058   -91  155.214942     -70.214942   -60.214942
2 -78.938046  36.001988   -69   52.424960      32.575040    42.575040
3 -78.938560  36.001236   -81   78.019994       6.980006    16.980006
4 -78.937970  36.002079   -65   69.698358      15.301642    25.301642


/var/folders/sd/_4gcyf1x307dwdn_5_g9jqy80000gn/T/ipykernel_20940/3282871818.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source['azimuth_end'][idx] = 90 - ((df_source['azimuth'][idx]) - ang_sep/2)
/var/folders/sd/_4gcyf1x307dwdn_5_g9jqy80000gn/T/ipykernel_20940/3282871818.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source['azimuth_start'][idx] = 90 - ((df_source['azimuth'][idx]) + ang_sep/2)
/var/folders/sd/_4gcyf1x307dwdn_5_g9jqy80000gn/T/ipykernel_20940/3282871818.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat